In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
import pomegranate as pg

## H10.2

In [12]:
cloudy = pg.DiscreteDistribution( {'F': 0.5, 'T': 0.5} )
sprinkler = pg.ConditionalProbabilityTable(
    [['F', 'F', 0.5],
     ['F', 'T', 0.5],
     ['T', 'F', 0.9],
     ['T', 'T', 0.1]], [cloudy])
rain = pg.ConditionalProbabilityTable(
    [['F', 'F', 0.8],
     ['F', 'T', 0.2],
     ['T', 'F', 0.2],
     ['T', 'T', 0.8]], [cloudy])
wet_grass = pg.ConditionalProbabilityTable(
    [['F', 'F', 'F', 1.0],
     ['F', 'F', 'T', 0.0],
     ['T', 'F', 'F', 0.1],
     ['T', 'F', 'T', 0.9],
     ['F', 'T', 'F', 0.1],
     ['F', 'T', 'T', 0.9],
     ['T', 'T', 'F', 0.01],
     ['T', 'T', 'T', 0.99]], [sprinkler, rain])

In [13]:
state_cloudy = pg.State(cloudy, name='Cloudy')
state_sprinkler = pg.State(sprinkler, name='Sprinkler')
state_rain = pg.State(rain, name='Rain')
state_wet_grass = pg.State(wet_grass, name='Wet Grass')

In [15]:
network = pg.BayesianNetwork()
network.add_states(state_cloudy, state_sprinkler, state_rain, state_wet_grass)
network.add_edge(state_cloudy, state_sprinkler)
network.add_edge(state_cloudy, state_rain)
network.add_edge(state_sprinkler, state_wet_grass)
network.add_edge(state_rain, state_wet_grass)
network.bake()

In [31]:
observations = {'Wet Grass': 'T'}

beliefs = network.predict_proba(observations)
print 'Probabilities for...'
for state, belief in zip(network.states, beliefs):
    print '{}: {}'.format(state.name, belief.parameters)

Probabilities for...
Cloudy: [{'T': 0.6035588437850328, 'F': 0.3964411562149671}]
Sprinkler: [{'T': 0.4040418053001052, 'F': 0.5959581946998947}]
Rain: [{'T': 0.7840742595572503, 'F': 0.21592574044274965}]
Wet Grass: [{'T': 1.0, 'F': 0.0}]


The probability that the sprinkler was active after observing that the grass is wet is 0.40.

In [32]:
observations = {'Wet Grass': 'T', 'Rain': 'T'}

beliefs = network.predict_proba(observations)
print 'Probabilities for...'
for state, belief in zip(network.states, beliefs):
    print '{}: {}'.format(state.name, belief.parameters)

Probabilities for...
Cloudy: [{'T': 0.7937131630648331, 'F': 0.20628683693516692}]
Sprinkler: [{'T': 0.1944990176817289, 'F': 0.8055009823182712}]
Rain: [{'T': 1.0, 'F': 0.0}]
Wet Grass: [{'T': 1.0, 'F': 0.0}]


With the additional observation of rain, the probability decreases to 0.19.

## H10.3

In [34]:
burglary = pg.DiscreteDistribution( {'F': 0.99, 'T': 0.01} )
earthquake = pg.DiscreteDistribution( {'F': 1.-1e-6, 'T': 1e-6} )
radio = pg.ConditionalProbabilityTable(
    [['F', 'F', 1.],
     ['F', 'T', 0.],
     ['T', 'F', 0.],
     ['T', 'T', 1.]], [earthquake])
alarm = pg.ConditionalProbabilityTable(
    [['F', 'F', 'F', 0.999],
     ['F', 'F', 'T', 0.001],
     ['F', 'T', 'F', 0.59],
     ['F', 'T', 'T', 0.41],
     ['T', 'F', 'F', 0.05],
     ['T', 'F', 'T', 0.95],     
     ['T', 'T', 'F', 0.02],
     ['T', 'T', 'T', 0.98]], [burglary, earthquake])

In [35]:
state_burglary = pg.State(burglary, name='Burglary')
state_earthquake = pg.State(earthquake, name='Earthquake')
state_radio = pg.State(radio, name='Radio')
state_alarm = pg.State(alarm, name='Alarm')

In [36]:
network = pg.BayesianNetwork()
network.add_states(state_burglary, state_earthquake, state_radio, state_alarm)
network.add_edge(state_burglary, state_alarm)
network.add_edge(state_earthquake, state_alarm)
network.add_edge(state_earthquake, state_radio)
network.bake()

In [37]:
observations = {}

beliefs = network.predict_proba(observations)
print 'Probabilities for...'
for state, belief in zip(network.states, beliefs):
    print '{}: {}'.format(state.name, belief.parameters)

Probabilities for...
Burglary: [{'T': 0.010000000000000002, 'F': 0.99}]
Earthquake: [{'T': 1.0000000000000004e-06, 'F': 0.999999}]
Radio: [{'T': 1.0000000000000004e-06, 'F': 0.999999}]
Alarm: [{'T': 0.01049040521, 'F': 0.98950959479}]


P(A=t) = 0.01

In [38]:
observations = {'Radio': 'T'}

beliefs = network.predict_proba(observations)
print 'Probabilities for...'
for state, belief in zip(network.states, beliefs):
    print '{}: {}'.format(state.name, belief.parameters)

Probabilities for...
Burglary: [{'T': 0.010000000000000004, 'F': 0.99}]
Earthquake: [{'T': 1.0, 'F': 0.0}]
Radio: [{'T': 1.0, 'F': 0.0}]
Alarm: [{'T': 0.4156999999999999, 'F': 0.5843}]


P(A=t|R=t) = 0.42

In [39]:
observations = {'Alarm': 'T'}

beliefs = network.predict_proba(observations)
print 'Probabilities for...'
for state, belief in zip(network.states, beliefs):
    print '{}: {}'.format(state.name, belief.parameters)

Probabilities for...
Burglary: [{'T': 0.9055894514869746, 'F': 0.09441054851302548}]
Earthquake: [{'T': 3.962668664159258e-05, 'F': 0.9999603733133584}]
Radio: [{'T': 3.962668664159258e-05, 'F': 0.9999603733133584}]
Alarm: [{'T': 1.0, 'F': 0.0}]


P(B=t|A=t) = 0.91

In [41]:
observations = {'Alarm': 'T', 'Radio': 'T'}

beliefs = network.predict_proba(observations)
print 'Probabilities for...'
for state, belief in zip(network.states, beliefs):
    print '{}: {}'.format(state.name, belief.parameters)

Probabilities for...
Burglary: [{'T': 0.02357469328842916, 'F': 0.9764253067115708}]
Earthquake: [{'T': 1.0, 'F': 0.0}]
Radio: [{'T': 1.0, 'F': 0.0}]
Alarm: [{'T': 1.0, 'F': 0.0}]


P(B=t|A=t, R=t) = 0.024